In [3]:
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt
import os
import scipy.optimize as opt

In [4]:
os.listdir()

['.ipynb_checkpoints',
 'Anamoly detection.ipynb',
 'ex8data1.mat',
 'ex8data2.mat',
 'ex8_movieParams.mat',
 'ex8_movies.mat',
 'Recommender system.ipynb']

In [5]:
data1 = loadmat('ex8_movieParams.mat')

In [6]:
data1.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [7]:
X = data1['X']
theta = data1['Theta']
num_users = data1['num_users']
num_movies = data1['num_movies']
num_features = data1['num_features']

In [8]:
print("Shape of X: ",X.shape)
print("Shape of theta: ",theta.shape)
print("Num of movies: ",num_movies)
print("Num of users: ",num_users)
print("Num of features: ",num_features)

Shape of X:  (1682, 10)
Shape of theta:  (943, 10)
Num of movies:  [[1682]]
Num of users:  [[943]]
Num of features:  [[10]]


In [9]:
data2 = loadmat('ex8_movies.mat')

In [10]:
data2.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])

In [11]:
y = data2['Y']
R = data2['R']

In [12]:
print(y.shape)
print(R.shape)


(1682, 943)
(1682, 943)


In [21]:
def  costfunction(X,y,theta,R,lambd):
    pred = np.dot(X,theta.T)
    err = (pred - y)
    j = (1/2)*np.sum((err**2)*R) + (lambd/2)*np.sum(theta**2)
    return j

In [22]:
num_users = 4
num_movies = 5
num_features = 3
X_test = X[:num_movies,:num_features]
theta_test = theta[:num_users,:num_features]
y_test = y[:num_movies,:num_users]
R_test = R[:num_movies,:num_users]
j = costfunction(X_test,y_test,theta_test,R_test,0)
print("J test: ",j)

J test:  22.224603725685675


In [36]:
def gradient(X,theta,y,R,lambd):
    
    nm ,nu = y.shape
    nf = X.shape[1]
    theta = theta.reshape((nu,nf))
    pred = np.dot(X,theta.T)

    theta[0] = np.dot(((pred-y)*R).T,X)[0]
    
    theta[1:] = np.dot(((pred-y)*R).T,X)[1:] + lambd*theta[1:]
    return theta
initial_theta = np.linspace()
print(initial_theta[:10])
grd = gradient(X,initial_theta,y,R,0)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [67]:
def randominit(m,n):
    theta = np.random.uniform(low=-0.12,high=0.12,size=(m,n))
    return theta

In [ ]:
def coficost(X,theta,y,R,lambd):
    
    nm,nu = y.shape
    nf = theta.shape[1]
    
    X = X.reshape((nu,nf))
    
    pred = np.dot(X,theta.T)
    
    j = (1/2)*np.sum(((pred-y)**2)*R) + lambd/2*np.sum(X**2)
    
    return j

    
    

In [44]:
def cofigradient(param,y,R,lambd):
    
    nm,nu = y.shape
    nf = theta.shape[1]
    
    X = X.reshape((nu,nf))
    
    pred = np.dot(X,theta.T)

    X[0] = np.dot(((pred-y)*R),theta)[0]
    
    X[1:] = np.dot((pred-y)*R,theta)[1:] + lambd*X[1:]
   
    return X
    
    

In [96]:
def colaborative_cost(param,y,R,lambd):
    
    nm,nu = y.shape
    nf=10
    X = param[:nm*nf].reshape((nm,nf))

    
    theta = param[nm*nf:].reshape((nu,nf))
  
    nm,nu = y.shape
    
    pred = np.dot(X,theta.T)
    
    err = pred-y
    
    j = (.5 * np.sum(((np.dot(theta,X.T).T - y) * R)**2) + ((lambd / 2) * np.sum(theta**2)) +((lambd / 2) * np.sum(X**2)))
    #j = (1/2)*np.sum((err**2)*R) + lambd/2*np.sum(X**2) + lambd/2*np.sum(theta**2)
    
    #X = np.dot((pred-y)*R,theta) + lambd*X
    X_grad = np.zeros_like(X)
    for i in range(nm):
        idx = np.where(R[i,:]==1)[0] # users who have rated movie i
        temp_theta = theta[idx,:] # parameter vector for those users 
        temp_Y = y[idx, :] # ratings given to movie i
        X_grad[i,:] = np.sum(np.dot(np.dot(temp_theta, X[i, :]) - temp_Y.T,temp_theta) + lambd*X[i,:], axis=0)
    
    Theta_grad = np.zeros_like(theta)
    for j in range(nu):
        idx = np.where(R[:,j]==1)[0]
        temp_X = X[idx,:]
        temp_Y = y[idx,j]
        Theta_grad[j,:] = np.sum(np.dot(np.dot(theta[j], temp_X.T) -temp_Y, temp_X) + lambd*theta[j], axis=0)
    
    #theta = np.dot(((pred-y)*R).T,X) + lambd*theta
    
    param = np.append(X_grad.ravel(),Theta_grad.ravel())
    
    return j,param


    

In [105]:
nm , nu = y.shape
nf = 10
lambd=10
#X = randominit(nm,nf)
#theta = randominit(nu,nf)

print(param.shape)
X = np.random.normal(size=(nm,nf))
theta = np.random.normal(size=(nu, nf))
param = np.append(X.ravel(),theta.ravel())

(26250,)


In [106]:
param_f ,nf,rc = opt.fmin_tnc(func=colaborative_cost,x0=param,args=(y,R,lambd))

In [107]:
X = param_f[:nm*10].reshape((nm,10))
theta = param_f[nm*10:].reshape((nu,10))

In [108]:
predd = np.dot(X,theta.T)

In [109]:
predd[:10]

array([[ 1.85956671,  2.55943224, -0.18868469, ..., -2.75528884,
        -4.23265657,  3.52285647],
       [-0.02784054, -0.66593173, -0.58768951, ..., -1.36126634,
         2.19354252, -4.10130248],
       [-0.23527686,  8.84613919,  2.90149632, ..., -1.82969216,
         0.6259907 , -3.80340551],
       ...,
       [ 0.35013874,  0.8180158 ,  4.19141377, ..., -2.93001753,
        -4.92920152, -1.24102463],
       [ 3.74481626, -1.16873027, -1.82124318, ...,  8.32087694,
         2.21522502, -0.41875914],
       [ 1.29727744,  5.7097198 ,  2.85604056, ...,  0.20991207,
         1.30565186, -4.82947211]])